# **Image segmentation – Basics**

<div style="color:#777777;margin-top: -15px;">
<b>Author</b>: Norman Juchler |
<b>Course</b>: MSLS CO4 |
<b>Version</b>: v1.2 <br><br>
<!-- Date: 16.04.2025 -->
<!-- Comments: Text refactored -->
</div>

In this notebook on segmentation, we will explore different approaches to segment hematological images. As a first step, we will attempt to segment the cells using simple thresholding techniques.

Several of the concepts discussed here are also covered in this insightful tutorial for the ImageJ/Fiji plugin [MorphoLibJ](https://imagej.net/plugins/morpholibj), which you may find helpful for further reference.


---

## **Preparations**

Let's begin with the usual preparatory steps...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import PIL
from pathlib import Path

# Jupyter / IPython configuration:
# Automatically reload modules when modified
%load_ext autoreload
%autoreload 2

# Enable vectorized output (for nicer plots)
%config InlineBackend.figure_formats = ["svg"]

# Inline backend configuration
%matplotlib inline

# Enable this line if you want to use the interactive widgets
# It requires the ipympl package to be installed.
#%matplotlib widget

import sys
sys.path.insert(0, "../")
import tools

We will use the same images that were used in the previous notebook on preprocessing:

In [ ]:
# Read in the data
img1 = cv.imread("../data/images/hematology-baso1.jpg", cv.IMREAD_COLOR)
img2 = cv.imread("../data/images/hematology-baso2.jpg", cv.IMREAD_COLOR)
img3 = cv.imread("../data/images/hematology-blast1.jpg", cv.IMREAD_COLOR)

img1 = cv.cvtColor(img1, cv.COLOR_BGR2RGB)
img2 = cv.cvtColor(img2, cv.COLOR_BGR2RGB)
img3 = cv.cvtColor(img3, cv.COLOR_BGR2RGB)

tools.show_image_chain([img1, img2, img3], titles=["img1", "img2", "img3"])

---

## **Method 1: Thresholding**

We can segment images using basic thresholding techniques. In this example, we explore several thresholding methods available in OpenCV:

- **Simple thresholding**: Use [`cv.threshold()`](https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57)  
  (with flags `cv.THRESH_BINARY` or `cv.THRESH_BINARY_INV`)
- **Adaptive thresholding**: Use [`cv.adaptiveThreshold()`](https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#ga72b913f352e4a1b1b397736707afcde3)
- **Otsu's thresholding** : Use [`cv.threshold()`](https://docs.opencv.org/4.x/d7/d1b/group__imgproc__misc.html#gae8a4a146d1ca78c626a53577199e9c57)  
  (with flags `cv.THRESH_BINARY + cv.THRESH_OTSU`)

Thresholding segments pixels into foreground and background based on their intensity values, making it a form of *binary* segmentation*. The algorithm compares pixel intensities to a threshold value: Pixel values larger than the threshold are classified as *foreground*, pixels smaller or equal than the threshold are *background*.

The threshold can be manually defined or automatically determined (e.g., by Otsu’s method).

As preparation, please review the following OpenCV documentation on thresholding methods:  
[https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html](https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html)


In [ ]:
######################
###    EXCERISE    ###
######################

# Choose here the image to work with
img = img1

# 1) Summarize the three different thresholding techniques in own words. Which methods
#    use a global threshold, which ones apply a local threshold? 

# 2) Develop a strategy to segment the white blood cells (purple), the red blood cells 
# (red) and the background (white/gray). You may want to exploit the fact that we have
# colors to work with:
tools.show_image_chain([img[:,:,0], img[:,:,1], img[:,:,2]], titles=["R", "G", "B"])

# 3) Identify the different regions using thresholding
mask_wbc = ...
mask_rbc = ...
mask_bg = ...

# 4) Visualize the masks. Idea: combine the three masks into an RGB image
mask_seg = ...

# 5) Discuss your results. What could be improved? What are the limitations of this 
#    approach? Are the masks mutually exclusive? Are they accurate?



---


## **Method 2: Color clustering**

Instead of segmenting the image into foreground and background, we can attempt to classify different regions based on color similarity. A common approach for this is the [K-means clustering](https://en.wikipedia.org/wiki/K-means_clustering) algorithm. K-means classifies pixels into a predefined number of clusters based on their color values. Similarity is typically measured using a (Euclidean or non-Euclidean) distance between pixel values. The algorithm operates iteratively:

1. Assign each pixel to the nearest cluster center.
2. Update each cluster center as the mean of the pixels assigned to it.
3. Repeat until the cluster centers converge.

Here is a helpful [visualization](https://www.naftaliharris.com/blog/visualizing-k-means-clustering/) of how K-means clustering works.


**Preparation:** Before you begin, check out these two tutorials:
- Jason Brownlee (Machine Learning Mastery) on [color quantization with K-means](https://machinelearningmastery.com/k-means-clustering-in-opencv-and-application-for-color-quantization/)
- Shubhang Agrawal on [image segmentation using K-means clustering](https://medium.com/swlh/image-segmentation-using-k-means-clustering-46a60488ae71). (The tutorial has a few flaws, please excuse). 


<!-- 
Resources:
# Nice way of depicting the bars
https://pyimagesearch.com/2014/05/26/opencv-python-k-means-color-clustering/
# OpenCV
https://docs.opencv.org/3.4/d1/d5c/tutorial_py_kmeans_opencv.html
# Machine Learning Mastery
https://machinelearningmastery.com/k-means-clustering-in-opencv-and-application-for-color-quantization/
# Watershed
https://docs.opencv.org/4.x/d3/db4/tutorial_py_watershed.html
# Segmentation with Skimage 
https://github.com/ipython-books/cookbook-2nd-code/blob/master/chapter11_image/03_segmentation.ipynb
# Combination between thresholding and color clustering
https://towardsdatascience.com/image-color-segmentation-by-k-means-clustering-algorithm-5792e563f26e
-->

In [ ]:
######################
###    EXCERISE    ###
######################

# Choose here the image to work with
img = img1

# 1) Reshape the color pixels into a Mx3 matrix (M: number of pixels)
#    and convert the data type to float32.
data = img.reshape(-1, 3).astype(np.float32)

# 2) Apply the K-means algorithm to the data. Use the cv.kmeans function.
#    Choose the number of clusters K=3.
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 3
ret, label, centers = cv.kmeans(data, K, None, criteria, 10, cv.KMEANS_RANDOM_CENTERS)
# label contains the cluster index for each pixel
# centers contains the cluster centers (colors!)

# 3) Reshape and convert the data back to uint8
img_seg = ...

# 4) Visualize the segmented image
tools.show_image_pair(img, img_seg, title1="Original", title2="Segmented");

# 5) Repeat the process for a different color space (e.g. HSV)
#    Is the clustering more robust? Why? When does this approach fail?


---

## **Method 3: Watershed algorithm for segmentation**

The watershed algorithm is a powerful tool for image segmentation, particularly useful for separating complex or overlapping structures. It is based on the concept of watershed lines, which define boundaries between different regions in an image. The algorithm works by conceptually "flooding" the image from predefined seed points. As water spreads from each seed region, it continues flowing until it meets water from a neighboring region.  
The boundaries where the regions meet are defined as the watershed lines, effectively separating the regions. Watershed segmentation can be applied based on pixel intensity or color and is especially effective for images with intricate shapes and touching objects.

Our dataset has structural similarities to the image used in this [OpenCV watershed tutorial](https://docs.opencv.org/4.x/d3/db4/tutorial_py_watershed.html). We will now follow a simplified version of the tutorial to segment our images using the watershed method. The tutorial uses the following strategy:

### **Overview / Steps**

1. Convert the image to a binary mask using thresholding.
2. Apply morphological operations to reduce noise and help separate objects. These operations are also used to identify regions that likely represent the background.
3. Identify seed points for the watershed algorithm:
   - Apply the distance transform to the binary mask. This computes, for each pixel, the distance to the nearest background pixel (value 0).
   - Threshold the distance-transformed image to isolate blobs near the centers of objects of interest.
   - Use the connected components algorithm to label and enumerate the seed regions, creating a labeled mask.
   - Mark the background seed region (identified in step 2) with label `0`.
4. Apply the watershed algorithm to segment the regions.
5. Visualize the resulting segmentation.



### **Note: Morphological operations**

Morphological operations are a set of operations used to analyze and manipulate the shape of objects in an image. Although they are defined for various image types, they are most commonly applied to binary images. These operations involve a structuring element (kernel) that probes the image and modifies pixel values based on the interaction between the kernel and the image. The most common operations include dilation (expands shapes), erosion (shrinks shapes), opening (erosion followed by dilation), and closing (dilation followed by erosion). Morphological operations are useful for removing noise, separating objects, and connecting disjoint regions in an image. There is a separate notebook on morphological operations.

**Further reading**:  
- OpenCV documentation: [Link](https://docs.opencv.org/4.x/d9/d61/tutorial_py_morphological_ops.html)  
- Beautiful illustration of morphological operations: [Link](https://penny-xu.github.io/blog/mathematical-morphology)  
- Wikipedia article on mathematical morphology: [Link](https://en.wikipedia.org/wiki/Mathematical_morphology)  
- Blog post on morphological operations: [Link](https://towardsdatascience.com/7bcf1ed11756)

### **Note: Distance transform**
The distance transform is useful for various image processing tasks. It computes the distance from each pixel to the nearest boundary (i.e., the closest background pixel) in a binary image. Distance transforms are used for operations like skeletonization, shape analysis, and segmentation. The algorithm propagates distance values from boundary pixels inward, typically using a metric such as the Euclidean distance. It is computationally efficient and widely available in image processing libraries.

**Further reading**:  
- Application of distance transform with watershed: [Link](https://docs.opencv.org/3.4/d2/dbd/tutorial_distance_transform.html)

### **Note: Connected components**

Connected components are regions in a binary image where pixels are connected based on predefined neighborhood rules (e.g., 4-connectivity or 8-connectivity). This technique is used to identify individual objects in a segmentation mask. The algorithm labels each connected region with a unique integer value, allowing for further analysis such as counting or measuring properties of each component.

**Further reading**:  
- Wikipedia article on connected component labeling: [Link](https://en.wikipedia.org/wiki/Connected-component_labeling)



In [ ]:
######################
###    EXCERISE    ###
######################

img = img1

# Implement the approach lined out above. You can copy paste the code 
# from the above link and plug our image into it. Try to understand the
# code and the different steps. You may have to adjust the parameters
# to get a good segmentation result.

# https://docs.opencv.org/4.x/d3/db4/tutorial_py_watershed.html

...

---

## **AI driven segmentation**
Deep learning is increasingly used for image segmentation tasks, with the U-Net architecture still being one of the most popular choices. U-Net is a convolutional neural network specifically designed for biomedical image segmentation. 

A specialized U-Net-based tool for medical imaging is [nnU-Net](https://github.com/MIC-DKFZ/nnUNet). It features self-configuring preprocessing and postprocessing, allowing the network to automatically adapt to the characteristics of the input data. nnU-Net is available as a Python package and can be installed via pip.

Although machine learning and AI are not the core focus of this course, pre-trained models can still be applied to perform segmentation effectively. Unlike classical methods, deep learning models can learn features directly from the data and often generalize better to unseen examples. However, they require large labeled datasets for training, are more computationally demanding, and are often seen as "black boxes" – making it difficult to interpret their decisions.


```python
######################
###    EXCERISE    ###
######################
```

Visit the following resources and explore whether they could be useful for your own segmentation project:

- **Segment Anything** by Meta AI [Demo](https://segment-anything.com/demo), [Paper](https://arxiv.org/abs/2304.02643), [Code](https://github.com/facebookresearch/segment-anything) 
- **Huggingface**: Collection of public pre-trained models. [Link](https://huggingface.co/models).
  - Many models include a demo interface
  - Background removal with [RemBG](https://huggingface.co/spaces/KenjieDec/RemBG)
  - Another popular segmentation tool is [YOLO](https://huggingface.co/spaces/fcakyon/yolov8-segmentation) ([Code](https://huggingface.co/spaces/fcakyon/yolov8-segmentation))
  - To search the entire Huggingface database for models: [Link](https://huggingface.co/models)
- **TotalSegmentator** for anatomical CT (and MR) segmentation. [Demo](https://totalsegmentator.com/), [Paper](https://arxiv.org/abs/2208.05868), [Code](https://github.com/wasserth/TotalSegmentator)


We have now explored several approaches to image segmentation.  How well you can apply them will depend on your specific problem – and a bit of engineering skill. 😊


In [ ]:
######################
###    EXERCISE    ###
######################

# Try using one of the models listed above to segment the cells in the image.